<a href="https://colab.research.google.com/github/Glebych2/MIPT_home_work/blob/main/Environment_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
!pip install -q -U kaggle_environments

In [118]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [119]:
%%writefile rock_agent.py
#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [120]:
%%writefile copy_opponent.py
import random
#Example 
def copy_opponent(observation, configuration):   
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
        #return 2
    #initial step
    else:
        return random.randrange(0, configuration.signs)
        #return random.randrange(0, 2)

Overwriting copy_opponent.py


In [121]:
%%writefile paper_agent.py
#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


In [122]:
%%writefile scissors_agent.py
#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [123]:
%%writefile submission.py

import random

def nash_equilibrium_agent(observation, configuration):
    return random.randint(0, 2)

Overwriting submission.py


In [184]:
%%writefile reactionary.py
import random
from utils import get_score

last_react_action = None
def reactionary(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action


Overwriting reactionary.py


In [195]:
%%writefile counter_reactionary.py
import random
from utils import get_score

last_counter_action = None
def counter_reactionary(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

Overwriting counter_reactionary.py


In [197]:
%%writefile statistical.py
import random
from utils import get_score 

def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Writing statistical.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [136]:
evaluate(
    "rps", #environment to use - no need to change
    ["submission.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100, 'tieRewardThreshold': 2} #number of episodes 
)

[[6.0, -6.0]]

In [190]:
evaluate(
    "rps", #environment to use - no need to change
    ["reactionary.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100, 'tieRewardThreshold': 2} #number of episodes 
)

[[-99.0, 99.0]]

In [196]:
evaluate(
    "rps", #environment to use - no need to change
    ["counter_reactionary.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100, 'tieRewardThreshold': 2} #number of episodes 
)

[[-96.0, 96.0]]

In [215]:
evaluate(
    "rps", #environment to use - no need to change
    ["statistical.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100, 'tieRewardThreshold': 1} #number of episodes 
)

[[21.0, -21.0]]

In [129]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps":100} #number of episodes 
)

[[99.0, -99.0]]